In [11]:
# model.py
from mesa.space import MultiGrid
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import random

        
def compute_gini(model):
    agent_wealths = [agent.wealth for agent in model.schedule.agents]
    x = sorted(agent_wealths)
    N = model.num_agents
    B = sum( xi * (N-i) for i,xi in enumerate(x) ) / (N*sum(x))
    return (1 + (1/N) - 2*B)

class MoneyModel(Model):
    """A model with some number of agents."""
    def __init__(self, N, width, height):
        self.num_agents = N
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)
        self.running = True

        # Create agents
        for i in range(self.num_agents):
            a = MoneyAgent(i, self)
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = random.randrange(self.grid.width)
            y = random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))

        self.datacollector = DataCollector(
            model_reporters={"Gini": compute_gini},
            agent_reporters={"Wealth": lambda a: a.wealth})

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()


class MoneyAgent(Agent):
    """ An agent with fixed initial wealth."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealth = 1

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False)
        new_position = random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def give_money(self):
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        if len(cellmates) > 1:
            other = random.choice(cellmates)
            other.wealth += 1
            self.wealth -= 1

    def step(self):
        self.move()
        if self.wealth > 0:
            self.give_money()


In [12]:
from mesa.batchrunner import BatchRunner

fixed_params = {"width": 10,
                "height": 10, 
                "N": range(10, 500, 10)}

batch_run = BatchRunner(MoneyModel,
                        parameter_values=fixed_params,
                        iterations=5,
                        max_steps=100,
                        model_reporters={"Gini": compute_gini})
batch_run.run_all()





  0%|          | 0/245 [00:00<?, ?it/s]


  2%|▏         | 5/245 [00:00<00:05, 44.98it/s]


  3%|▎         | 8/245 [00:00<00:06, 36.66it/s]


  4%|▍         | 11/245 [00:00<00:07, 31.01it/s]


  5%|▌         | 13/245 [00:00<00:09, 25.17it/s]


  6%|▌         | 15/245 [00:00<00:10, 22.49it/s]


  7%|▋         | 17/245 [00:00<00:11, 19.01it/s]


  8%|▊         | 19/245 [00:00<00:13, 17.11it/s]


  9%|▊         | 21/245 [00:01<00:14, 15.32it/s]



  9%|▉         | 23/245 [00:01<00:17, 12.83it/s]


 10%|█         | 25/245 [00:01<00:18, 12.15it/s]


 11%|█         | 27/245 [00:01<00:19, 11.09it/s]


 12%|█▏        | 29/245 [00:01<00:20, 10.59it/s]


 13%|█▎        | 31/245 [00:02<00:21,  9.80it/s]


 13%|█▎        | 33/245 [00:02<00:23,  9.15it/s]


 14%|█▍        | 34/245 [00:02<00:24,  8.72it/s]


 14%|█▍        | 35/245 [00:02<00:24,  8.44it/s]


 15%|█▍        | 36/245 [00:02<00:26,  7.99it/s]


 15%|█▌        | 37/245 [00:02<00:27,  7.70it/s]


 16%|█▌        | 38/245 [00:03<00:27, 

 63%|██████▎   | 154/245 [00:41<00:45,  2.02it/s]


 63%|██████▎   | 155/245 [00:41<00:44,  2.01it/s]


 64%|██████▎   | 156/245 [00:42<00:44,  1.99it/s]


 64%|██████▍   | 157/245 [00:42<00:44,  1.97it/s]


 64%|██████▍   | 158/245 [00:43<00:44,  1.95it/s]


 65%|██████▍   | 159/245 [00:43<00:44,  1.94it/s]


 65%|██████▌   | 160/245 [00:44<00:44,  1.91it/s]


 66%|██████▌   | 161/245 [00:44<00:46,  1.83it/s]


 66%|██████▌   | 162/245 [00:45<00:45,  1.84it/s]


 67%|██████▋   | 163/245 [00:45<00:44,  1.85it/s]


 67%|██████▋   | 164/245 [00:46<00:43,  1.86it/s]


 67%|██████▋   | 165/245 [00:46<00:42,  1.86it/s]


 68%|██████▊   | 166/245 [00:47<00:42,  1.85it/s]


 68%|██████▊   | 167/245 [00:48<00:42,  1.84it/s]


 69%|██████▊   | 168/245 [00:48<00:41,  1.84it/s]


 69%|██████▉   | 169/245 [00:49<00:41,  1.84it/s]


 69%|██████▉   | 170/245 [00:49<00:40,  1.83it/s]


 70%|██████▉   | 171/245 [00:50<00:40,  1.81it/s]


 70%|███████   | 172/245 [00:50<00:40,  1.80it/s]


 71%|███████